In [1]:
import numpy as np
import pandas as pd
import auxil_ML as aux

In [2]:
if 1:
    import importlib
    importlib.reload(aux);


In [104]:
#cat_fn = 'data/3FGL_prob_catalog_3class_newfeats.csv'
#cat_fn = '../ML_3FGL_paper_extra/data/python files/catas/3fgl_multi_cata_newfeats.csv'
cat_fn = '../ML_3FGL_paper/SOM/3FGL_prob_catalog_3classes.csv'
cat_df3 = pd.read_csv(cat_fn, index_col=0)
cat_fn = '../ML_3FGL_paper/SOM/3FGL_prob_catalog_2classes.csv'
cat_df2 = pd.read_csv(cat_fn, index_col=0)

#folder = '../ML_3FGL_paper/SOM'
#cat_fn = '%s/3FGL_prob_catalog.csv' % folder
cat_fn = '../ML_3FGL_paper/SOM/3FGL_prob_catalog_2classes.csv'
cat_df_3fgl = pd.read_csv(cat_fn, index_col=0)
cat_fn = '%s/4FGL-DR2_prob_catalog_2classes.csv' % folder
cat_df_4fgl = pd.read_csv(cat_fn, index_col=0)

In [105]:
nclass = 3
classes = ['AGN', 'PSR', 'OTHER'][:nclass]
algs = ['RF', 'BDT', 'NN', 'LR']

In [106]:
assoc_3FGL_in_4FGL_mask = [(type(name) is str) for name in list(cat_df_4fgl['ASSOC_FGL'])]
index0 = [name for name in cat_df_4fgl.index[assoc_3FGL_in_4FGL_mask] 
         if cat_df_4fgl['ASSOC_FGL'].loc[name].startswith('3FGL') 
         and cat_df_4fgl['ASSOC_FGL'].loc[name] in cat_df_3fgl.index
         and cat_df_3fgl['Category_3FGL'].loc[cat_df_4fgl['ASSOC_FGL'].loc[name]] == 'UNAS']
index = [name for name in index0 if cat_df_4fgl['Category_4FGL'].loc[name] != 'UNAS']
index_2class = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] in ['PSR', 'AGN']]
index_3class = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] in ['PSR', 'AGN', 'OTHER']]
if nclass == 2:
    index = index_2class
elif nclass == 3:
    index = index_3class
index_unas = [name for name in index0 if cat_df_4fgl['Category_4FGL'].loc[name] == 'UNAS']
index_other = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] == 'OTHER']

assoc_3FGL_in_4FGL_df = pd.DataFrame(index=cat_df_4fgl['ASSOC_FGL'][index])
assoc_3FGL_in_4FGL_df['Category_4FGL'] = np.array(cat_df_4fgl['Category_4FGL'].loc[index])
psr_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'PSR')
agn_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'AGN')
other_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'OTHER')


In [107]:
print('Unas in 3FGL, assoc in 4FGL-DR2:', len(index))
print('Unas in 3FGL, unas in 4FGL-DR2:', len(index_unas))
print('Unas in 3FGL, AGN or PSR in 4FGL-DR2:', len(index_2class))
print('Unas in 3FGL, OTHER in 4FGL-DR2:', len(index_other))
print('Ratio of OTHER to assoc in 4FGL-DR2:', len(index_other)/len(index))

Unas in 3FGL, assoc in 4FGL-DR2: 339
Unas in 3FGL, unas in 4FGL-DR2: 303
Unas in 3FGL, AGN or PSR in 4FGL-DR2: 302
Unas in 3FGL, OTHER in 4FGL-DR2: 37
Ratio of OTHER to assoc in 4FGL-DR2: 0.10914454277286136


In [108]:
other_4fgl = sum(cat_df_4fgl['Category_4FGL'] == 'OTHER')
assoc_4fgl = sum(cat_df_4fgl['Category_4FGL'] != 'UNAS')
print('OTHER in 4FGL-DR2: %i' % other_4fgl)
print('Assoc in 4FGL-DR2: %i' % assoc_4fgl)
print('Fraction: %.4f' % (other_4fgl/assoc_4fgl))

OTHER in 4FGL-DR2: 346
Assoc in 4FGL-DR2: 4116
Fraction: 0.0841


In [109]:
cat_df_3fgl_i = cat_df_3fgl.loc[assoc_3FGL_in_4FGL_df.index]
cat_df3_i = cat_df3.loc[assoc_3FGL_in_4FGL_df.index]
cat_df2_i = cat_df2.loc[assoc_3FGL_in_4FGL_df.index]


In [110]:
psr_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'PSR')
agn_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'AGN')
mix_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'MIXED')

psr_mask_prob2 = (cat_df2_i['Category_Prob'] == 'PSR')
agn_mask_prob2 = (cat_df2_i['Category_Prob'] == 'AGN')
mix_mask_prob2 = (cat_df2_i['Category_Prob'] == 'MIXED')

psr_mask_prob3 = (cat_df3_i['Category_Prob'] == 'PSR')
agn_mask_prob3 = (cat_df3_i['Category_Prob'] == 'AGN')
other_mask_prob3 = (cat_df3_i['Category_Prob'] == 'OTHER')
mix_mask_prob3 = (cat_df3_i['Category_Prob'] == 'MIXED')

In [111]:
print('2-class classification in paper')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob))

2-class classification in paper
# PSR classified as PSR 23
# PSR classified as AGN 10
# PSR classified as mixed 12
# AGN classified as AGN 222
# AGN classified as PSR 5
# AGN classified as mixed 30
# OTHER classified as AGN 8
# OTHER classified as PSR 6
# OTHER classified as mixed 23


In [112]:
print('2-class classification')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob2 ))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob2 ))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob2 ))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob2 ))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob2 ))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob2 ))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob2))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob2))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob2))

2-class classification
# PSR classified as PSR 23
# PSR classified as AGN 10
# PSR classified as mixed 12
# AGN classified as AGN 222
# AGN classified as PSR 5
# AGN classified as mixed 30
# OTHER classified as AGN 8
# OTHER classified as PSR 6
# OTHER classified as mixed 23


In [113]:
print('3-class classification')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob3 ))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob3 ))
print('# PSR classified as OTHER', np.sum(psr_mask_4FGL & other_mask_prob3 ))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob3 ))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob3 ))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob3 ))
print('# AGN classified as OTHER', np.sum(agn_mask_4FGL & other_mask_prob3 ))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob3 ))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob3))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob3))
print('# OTHER classified as OTHER', np.sum(other_mask_4FGL & other_mask_prob3))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob3))

3-class classification
# PSR classified as PSR 17
# PSR classified as AGN 12
# PSR classified as OTHER 0
# PSR classified as mixed 16
# AGN classified as AGN 237
# AGN classified as PSR 2
# AGN classified as OTHER 1
# AGN classified as mixed 17
# OTHER classified as AGN 6
# OTHER classified as PSR 5
# OTHER classified as OTHER 8
# OTHER classified as mixed 18


In [114]:
cat_df_3fgl_i

,GLON,GLAT,ln(Energy_Flux100),ln(Unc_Energy_Flux100),ln(Signif_Curve),ln(Variability_Index),500MeV_Index,HR12,HR23,HR34,...,PSR_NN_STD_O,AGN_RF_O,AGN_RF_STD_O,PSR_RF_O,PSR_RF_STD_O,AGN_LR_O,AGN_LR_STD_O,PSR_LR_O,PSR_LR_STD_O,Category_Prob
ASSOC_FGL,,,,,,,,,,,,,,,,,,,,,
3FGL J0002.0-6722,310.138920,-49.061836,-26.432554,-27.904511,0.313862,3.831355,1.946061,0.091518,-0.522756,-0.422249,...,0.072763,0.994358,0.006044,0.005642,0.006044,0.974034,0.009505,0.025966,0.009505,AGN
3FGL J0004.2+0843,103.599490,-52.363342,-26.538473,-27.860311,-0.235299,3.814006,2.050442,-0.755126,-0.477670,0.000580,...,0.069112,0.925361,0.047492,0.074639,0.047492,0.994295,0.005208,0.005705,0.005208,AGN
3FGL J0006.2+0135,100.400505,-59.296867,-26.405393,-27.897549,-1.778628,3.819167,2.186900,-0.611432,-0.563140,-0.645471,...,0.067848,0.989319,0.009945,0.010681,0.009945,0.999908,0.000101,0.000092,0.000101,AGN
3FGL J0007.9+4006,113.977170,-22.007206,-26.355611,-27.811253,-0.020849,3.793005,2.275604,-0.236408,-0.785758,-0.459872,...,0.064161,0.976730,0.021334,0.023270,0.021334,0.984549,0.007552,0.015451,0.007552,AGN
3FGL J0008.3+1456,107.678590,-46.658096,-26.043447,-27.812366,0.449795,3.539210,2.269549,-0.494926,-0.545080,-0.666568,...,0.133714,0.883812,0.082757,0.116188,0.082757,0.613692,0.084189,0.386308,0.084189,AGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3FGL J1855.6+0753,40.430542,2.657492,-24.114420,-26.370483,0.829996,3.433813,2.601826,-0.714195,-0.782879,-0.610913,...,0.141321,0.392464,0.114526,0.607536,0.114526,0.068482,0.023215,0.931518,0.023215,MIXED
3FGL J2004.4+3338,70.671234,1.185360,-23.868170,-26.605729,1.214844,3.917865,2.296311,-0.502915,-0.698934,-0.649789,...,0.157375,0.572448,0.097583,0.427552,0.097583,0.179261,0.051608,0.820739,0.051608,MIXED
3FGL J2058.0+4347,85.006134,-1.229651,-24.140762,-26.781395,0.417416,3.763118,2.920238,-0.725957,-0.861170,-0.999806,...,0.106712,0.731951,0.090436,0.268049,0.090436,0.981365,0.036243,0.018635,0.036243,AGN


In [115]:
true_labels = assoc_3FGL_in_4FGL_df['Category_4FGL']
classif_dict = {'paper': cat_df_3fgl_i, '2 class': cat_df2_i, '3 class': cat_df3_i}
for cname, cdf in classif_dict.items():
    print(cname)
    if cname == 'paper':
        algs_loc = algs + [alg + '_O' for alg in algs]
    else:
        algs_loc = algs + [alg + '_O' for alg in algs]
    for alg in algs_loc:
        prob_labels = aux.get_prob_class(cdf, [alg], classes)['Category_Prob']
        acc_all = 100 * aux.accuracy(true_labels, prob_labels)
        #print(sum(prob_labels == 'OTHER'), sum(true_labels == 'OTHER'))
        print(sum(prob_labels == true_labels))
        print(sum((prob_labels == 'OTHER') & (prob_labels == true_labels)))
        print('%s, %.2f percent' % (alg, acc_all))

paper
275
0
RF, 81.12 percent
273
0
BDT, 80.53 percent
272
0
NN, 80.24 percent
273
0
LR, 80.53 percent
270
0
RF_O, 79.65 percent
277
0
BDT_O, 81.71 percent
263
0
NN_O, 77.58 percent
257
0
LR_O, 75.81 percent
2 class
275
0
RF, 81.12 percent
273
0
BDT, 80.53 percent
272
0
NN, 80.24 percent
273
0
LR, 80.53 percent
270
0
RF_O, 79.65 percent
277
0
BDT_O, 81.71 percent
263
0
NN_O, 77.58 percent
257
0
LR_O, 75.81 percent
3 class
288
12
RF, 84.96 percent
282
13
BDT, 83.19 percent
283
12
NN, 83.48 percent
282
15
LR, 83.19 percent
288
15
RF_O, 84.96 percent
289
16
BDT_O, 85.25 percent
277
15
NN_O, 81.71 percent
282
16
LR_O, 83.19 percent


In [88]:
true_labels

ASSOC_FGL
3FGL J0002.0-6722    AGN
3FGL J0004.2+0843    AGN
3FGL J0006.2+0135    AGN
3FGL J0007.9+4006    AGN
3FGL J0008.3+1456    AGN
                    ... 
3FGL J2035.0+3634    PSR
3FGL J2038.8-3613    PSR
3FGL J2039.6-5618    PSR
3FGL J2045.6-6838    PSR
3FGL J2310.1-0557    PSR
Name: Category_4FGL, Length: 302, dtype: object

In [30]:
np.sum((prob_labels == true_labels)) / len(true_labels)

0.831858407079646

In [17]:
cat_df2_i

,GLON,GLAT,ln(Energy_Flux100),ln(Unc_Energy_Flux100),ln(Signif_Curve),ln(Variability_Index),500MeV_Index,HR12,HR23,HR34,...,PSR_NN_STD_O,AGN_RF_O,AGN_RF_STD_O,PSR_RF_O,PSR_RF_STD_O,AGN_LR_O,AGN_LR_STD_O,PSR_LR_O,PSR_LR_STD_O,Category_Prob
ASSOC_FGL,,,,,,,,,,,,,,,,,,,,,
3FGL J0002.0-6722,310.138920,-49.061836,-26.432554,-27.904511,0.313862,3.831355,1.946061,0.091518,-0.522756,-0.422249,...,0.072763,0.994358,0.006044,0.005642,0.006044,0.974034,0.009505,0.025966,0.009505,AGN
3FGL J0004.2+0843,103.599490,-52.363342,-26.538473,-27.860311,-0.235299,3.814006,2.050442,-0.755126,-0.477670,0.000580,...,0.069112,0.925361,0.047492,0.074639,0.047492,0.994295,0.005208,0.005705,0.005208,AGN
3FGL J0006.2+0135,100.400505,-59.296867,-26.405393,-27.897549,-1.778628,3.819167,2.186900,-0.611432,-0.563140,-0.645471,...,0.067848,0.989319,0.009945,0.010681,0.009945,0.999908,0.000101,0.000092,0.000101,AGN
3FGL J0007.9+4006,113.977170,-22.007206,-26.355611,-27.811253,-0.020849,3.793005,2.275604,-0.236408,-0.785758,-0.459872,...,0.064161,0.976730,0.021334,0.023270,0.021334,0.984549,0.007552,0.015451,0.007552,AGN
3FGL J0008.3+1456,107.678590,-46.658096,-26.043447,-27.812366,0.449795,3.539210,2.269549,-0.494926,-0.545080,-0.666568,...,0.133714,0.883812,0.082757,0.116188,0.082757,0.613692,0.084189,0.386308,0.084189,AGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3FGL J1855.6+0753,40.430542,2.657492,-24.114420,-26.370483,0.829996,3.433813,2.601826,-0.714195,-0.782879,-0.610913,...,0.141321,0.392464,0.114526,0.607536,0.114526,0.068482,0.023215,0.931518,0.023215,MIXED
3FGL J2004.4+3338,70.671234,1.185360,-23.868170,-26.605729,1.214844,3.917865,2.296311,-0.502915,-0.698934,-0.649789,...,0.157375,0.572448,0.097583,0.427552,0.097583,0.179261,0.051608,0.820739,0.051608,MIXED
3FGL J2058.0+4347,85.006134,-1.229651,-24.140762,-26.781395,0.417416,3.763118,2.920238,-0.725957,-0.861170,-0.999806,...,0.106712,0.731951,0.090436,0.268049,0.090436,0.981365,0.036243,0.018635,0.036243,AGN


In [73]:
270./302

0.8940397350993378

In [103]:
288./339

0.8495575221238938